In [1]:
from homeharvest import scrape_property
from datetime import datetime
import pandas as pd

In [2]:
# Generate filename based on current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"HomeHarvest_{current_timestamp}.csv"

properties = scrape_property(
  location="New York City, NY",
  listing_type="for_sale",  # or (for_sale, for_rent, pending)
  past_days=30,  # sold in last 30 days - listed in last 30 days if (for_sale, for_rent)

  # property_type=['single_family','multi_family'],
  # date_from="2023-05-01", # alternative to past_days
  # date_to="2023-05-28",
  # foreclosure=True
  # mls_only=True,  # only fetch MLS listings
)
print(f"Number of properties: {len(properties)}")

# Export to csv
# properties.to_csv(filename, index=False)
# print(properties.head())

KeyboardInterrupt: 

In [18]:
def read_data(file_name):
    """
    Read the data from the file and return a Pandas DataFrame
    file_name: str, name of the file to read
    return: Pandas DataFrame
    """
    return pd.read_csv(file_name)

data = read_data("HomeHarvest_20250425_154044.csv")
display(data.head)

<bound method NDFrame.head of                                            property_url  property_id  \
0     https://www.realtor.com/realestateandhomes-det...   9970797479   
1     https://www.realtor.com/realestateandhomes-det...   9763769548   
2     https://www.realtor.com/realestateandhomes-det...   4373587091   
3     https://www.realtor.com/realestateandhomes-det...   3999961443   
4     https://www.realtor.com/realestateandhomes-det...   3948723737   
...                                                 ...          ...   
6410  https://www.realtor.com/realestateandhomes-det...   3904036492   
6411  https://www.realtor.com/realestateandhomes-det...   9434052616   
6412  https://www.realtor.com/realestateandhomes-det...   4377385282   
6413  https://www.realtor.com/realestateandhomes-det...   4901716421   
6414  https://www.realtor.com/realestateandhomes-det...   4246170586   

      listing_id   mls                                mls_id    status  \
0     2981007048  STNY         

In [17]:
def pre_process(data):
    """
    Preprocess the data
    data: Pandas DataFrame
    return: Pandas DataFrame
    """
    data = data.dropna(subset=['sqft', 'list_price'])
    data["nearby_schools"] = data["nearby_schools"].fillna("")
    data["half_baths"] = data["half_baths"].fillna(0)
    data["parking_garage"] = data["parking_garage"].fillna(0)
    data["full_baths"] = data["full_baths"].fillna(0)
    data["beds"] = data["beds"].fillna(0)
    data["stories"] = data["stories"].fillna(1)
    data["year_built"] = data["year_built"].fillna(data["year_built"].median())
    data["num_schools"] = data["nearby_schools"].apply(lambda x : x.count(",") + 1 if x else 0)
    trimColumns = ['property_url','property_id','listing_id','mls','mls_id','status','text','full_street_line','street','unit','city','state','zip_code','days_on_mls','list_price_min','list_price_max','sold_price','last_sold_date','assessed_value','estimated_value','tax','tax_history','new_construction','lot_sqft','price_per_sqft','latitude','longitude','neighborhoods','county','fips_code','hoa_fee','agent_id','agent_name','agent_email','agent_phones','agent_mls_set','agent_nrds_id','broker_id','broker_name','builder_id','builder_name','office_id','office_mls_set','office_name','office_email','office_phones','primary_photo','alt_photos', 'nearby_schools', 'list_date']
    data = data.drop(trimColumns, axis = 1)
    data = pd.get_dummies(data, columns=['style'], drop_first=True)
    return data
clean_data = pre_process(data)
display(clean_data)

C:\Users\david\AppData\Local\Temp\ipykernel_44660\2312413594.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["nearby_schools"] = data["nearby_schools"].fillna("")
C:\Users\david\AppData\Local\Temp\ipykernel_44660\2312413594.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["half_baths"] = data["half_baths"].fillna(0)
C:\Users\david\AppData\Local\Temp\ipykernel_44660\2312413594.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,beds,full_baths,half_baths,sqft,year_built,list_price,stories,parking_garage,num_schools,style_CONDOP,style_CONDOS,style_COOP,style_MOBILE,style_MULTI_FAMILY,style_SINGLE_FAMILY,style_TOWNHOMES
0,3.0,2.0,0.0,1060.0,1901.0,180000.0,1.0,0.0,2,False,False,False,False,False,True,False
2,0.0,1.0,0.0,877.0,1939.0,300000.0,1.0,0.0,2,False,False,True,False,False,False,False
4,2.0,1.0,1.0,816.0,1979.0,498888.0,2.0,0.0,3,False,True,False,False,False,False,False
6,2.0,2.0,0.0,1370.0,1929.0,1675000.0,1.0,0.0,1,False,False,True,False,False,False,False
7,3.0,3.0,0.0,2041.0,2006.0,5595000.0,2.0,0.0,1,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,6.0,5.0,2.0,4441.0,1860.0,7750000.0,1.0,0.0,1,False,False,False,False,False,False,True
6411,1.0,1.0,0.0,728.0,1960.0,279000.0,1.0,0.0,2,False,False,True,False,False,False,False
6412,5.0,5.0,0.0,2400.0,1956.0,1900000.0,2.0,0.0,3,False,False,False,False,True,False,False
6413,1.0,1.0,0.0,750.0,1949.0,289000.0,1.0,0.0,2,False,False,True,False,False,False,False
